In [2]:
import cvxpy as cp
import numpy as np

op_demand = np.array([7000, 5500, 4500, 3000], dtype=float)
prod_demand = np.array([2200, 1600, 1400, 1800, 1200], dtype=float)
machine_cost = np.array([320000, 240000, 280000, 300000], dtype=float)  # M
carrier_cost = np.array([95000,  85000,  75000], dtype=float)           # MH
budget = 10000000
C_op = np.array([
    [ 8, 11,  9, 14],
    [10,  9,  7, 12],
    [12,  6,  5,  8],
    [15, 13, 11,  6],
], dtype=float)
C_hand = np.array([
    [ 8, 10, 11],   # b
    [16,  4, 13],   # h
    [11,  9,  9],   # s
    [10,  8,  6],   # l
    [14, 12,  8],   # p
], dtype=float)

t_op   = 8.0 / C_op   
t_hand = 1.0 / C_hand
T_avail = 750.0

v = cp.Variable(4, integer=True, nonneg=True, name="machines")
w = cp.Variable(3, integer=True, nonneg=True, name="carriers")

x = cp.Variable((4, 4), nonneg=True, name="op_assignment")
y = cp.Variable((5, 3), nonneg=True, name="hand_assignment")

cons = []
cons += [cp.sum(x, axis=1) == op_demand]

for j in range(4):
    cons += [cp.sum(cp.multiply(x[:, j], t_op[:, j])) <= T_avail * v[j]]

cons += [cp.sum(y, axis=1) == prod_demand]

for n in range(3):
    cons += [cp.sum(cp.multiply(y[:, n], t_hand[:, n])) <= T_avail * w[n]]

cons += [machine_cost @ v + carrier_cost @ w <= budget]

total_purchase = machine_cost @ v + carrier_cost @ w
total_oper     = cp.sum(cp.multiply(x, C_op))
total_hand     = cp.sum(cp.multiply(y, C_hand))
objective = cp.Minimize(total_purchase + total_oper + total_hand)

prob = cp.Problem(objective, cons)
prob.solve(solver=cp.MOSEK)

def iv(a):
    return np.round(a).astype(int)

print("Status:", prob.status)
print(f"Optimal total cost: ${prob.value:.2f}")
print("Machines v (M1, M2, M3, M4):", iv(v.value))
print("Carriers w (MH1, MH2, MH3):", iv(w.value))


Status: optimal
Optimal total cost: $5460010.95
Machines v (M1, M2, M3, M4): [4 8 0 6]
Carriers w (MH1, MH2, MH3): [0 1 1]
